In [ ]:
# %load 10_2022_load_config.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import sys
import plotly.express as px
import plotly.io as pio
import yaml

sns.set_context("notebook", font_scale=1.4)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
plt.rcParams["figure.figsize"] = (16, 12)
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.autolayout'] = False
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x))


config_file = "10_2022_analysis.yaml"
with open(config_file) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    configs = yaml.load(file, Loader=yaml.FullLoader)
    
# Run on server:
root = Path(configs['root'])
scratchDir = root/configs['scratchDir']
figuresDir = root/configs['figuresDir']
libraries = configs['libraries']
resultsDir = root/configs['unenrichedDir']

alphabetClrs = px.colors.qualitative.Alphabet
clrs = ["#f7ba65", "#bf4713", "#9c002f", "#d73d00", "#008080", "#004c4c"]
colors = {'grey': alphabetClrs[8], 
        'light_yellow': clrs[0],
        'darko': clrs[1],
        'maroon':clrs[2],
        'brighto': clrs[3],
        'teal':clrs[4],
        'darkteal':clrs[5]
       }

In [ ]:
import pyranges as pr

In [ ]:
resFiles = [f for f in (root/resultsDir).glob("*_rra_results.csv")]
results = pd.concat([pd.read_csv(f).assign(library=f.stem.split("_rra")[0]) for f in resFiles])
results['fdr'] = results[['neg_selection_fdr', 'pos_selection_fdr']].min(axis=1)
results = results[~results.Name.str.contains(":")]
gff = pr.read_gff3(configs['gff'], as_df=True)
gene_info = gff[gff.Feature == 'gene'][['Chromosome', 'Start', 'End', 'Name', 'locus_tag']]
gene_info['Chromosome'] = gene_info.Chromosome.astype('str')
prot_info = gff[gff.Feature == 'CDS'][['protein_id', 'locus_tag']]

In [ ]:
ann = pd.read_table(configs["eggnog"], comment='#').iloc[:, [0, 8, 10,11, 12,13,14,15,16,17,18,19]]
ann.columns = ['protein_id', 'Prefered_Name', 'EC', 'KEGG_ko', 'KEGG_Pathway',
       'KEGG_Module', 'KEGG_Reaction', 'KEGG_rclass', 'BRITE', 'KEGG_TC',
       'CAZy', 'BiGG_Reaction']
ann_info = (prot_info.merge(gene_info, on='locus_tag', how='outer')
            .merge(ann, on='protein_id', how='outer'))

In [ ]:
fdf = results.merge(ann_info, on='Name', how='outer')

In [ ]:
results.head()

In [ ]:
lib102 = results[results.library == 'library_10_2']
lib132 = results[results.library == 'library_13_2']

In [ ]:
lib102[(lib102.LFC < -1) & (lib102.fdr< 0.01)].sample(10)

In [ ]:
lib132[(lib132.LFC < -1) & (lib132.fdr< 0.01)]

In [ ]:
results[(abs(results.LFC) > 1) & (results.fdr < 0.01)].groupby('library').Name.nunique()

In [ ]:
resSum = (results.groupby(['Name', 'contrast'])
          .agg({'LFC':['mean', 'median', 'std'], 'library':['nunique']})
          .reset_index())
resSum.columns = ['Name', 'contrast', 'LFC_mean', 'LFC_median', 'LFC_std', 'library_count']

In [ ]:
fdf = fdf.merge(resSum, on=['Name', 'contrast'], how='left')

In [ ]:
fdf.to_csv(root/resultsDir/'25-10-22-unenriched-annotated-results.csv', index=False)

In [ ]:
fdf.sample(10)

In [ ]:
resSum[(resSum.library_count > 3) & (resSum.LFC_mean < -1)]